# ⚠️ 免責事項と利用上の注意 (Disclaimer & Limitations)
本ツールは、音響学の理論に基づく統計的な推定であり、その精度を保証するものではありません。発話内容、マイクの品質、環境ノイズ、発声の癖によって結果は大きく変動します。

エンターテイメント目的の利用: 本ツールの推定結果は、あくまで音響学的な参考値として、エンターテイメントの範囲でお楽しみください。 正確な身長を示すものではありません。

利用の制限: 本ツールの使用、またはその結果の利用によって生じたいかなる損害についても、製作者(yakimi)は一切の責任を負いません。 本ツールを、個人の特定、他者の名誉毀損、またはその他の違法な目的・犯罪行為に利用することを固く禁じます。

ライセンスに基づく無保証: 本プロジェクトはMITライセンスに基づき提供されており、「現状有姿のまま (AS IS)」で提供されます。製作者は、本ソフトウェアの動作、特定目的への適合性、または結果の正確性について、一切の保証を行いません。

In [ ]:
# 音声変換・処理に必要なライブラリをインストール
!pip install praat-parselmouth numpy pydub

from google.colab import files
import warnings

# @title ステップ1: 性別を選択してください
# フォルマントの探索範囲を決定するために性別が必要です
gender = "\u7537\u6027 (Male)" # @param ["男性 (Male)", "女性 (Female)"]
print(f"性別が '{gender}' に設定されました。")

print("\nステップ2: 身長推定に使用する音声ファイル（.wav, .mp3, .m4aなど）をアップロードしてください。")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    uploaded = files.upload()

if not uploaded:
    print("\nファイルがアップロードされませんでした。")
else:
    original_audio_path = list(uploaded.keys())[0]
    print(f"\nファイル '{original_audio_path}' がアップロードされました。")

In [ ]:
# @title ステップ2: 身長を計算します


import parselmouth
import numpy as np
import os
from pydub import AudioSegment
import traceback # エラー詳細表示用

# --- 共通ユーティリティ ---

def convert_audio_to_target_format(
    original_path: str,
    target_path: str = "converted_audio.wav"
) -> str | None:
    """pydubを使い、音声を 44.1kHz, 16bit, モノラル, WAV に変換します。"""
    try:
        audio = AudioSegment.from_file(original_path)
        audio = audio.set_frame_rate(44100).set_channels(1).set_sample_width(2)
        audio.export(target_path, format="wav")
        return target_path
    except Exception as e:
        print(f"音声変換中にエラーが発生しました: {e}")
        return None

# --- モデル1: F0（基本周波数）アプローチ ---

def get_robust_f0(snd: parselmouth.Sound, selected_gender: str) -> float | None:
    """
    parselmouth.Sound オブジェクトから頑健な平均F0を抽出します。
    """
    if selected_gender == "男性 (Male)":
        pitch_floor, pitch_ceiling = 75.0, 300.0
    else: # 女性 (Female)
        pitch_floor, pitch_ceiling = 150.0, 500.0

    try:
        pitch = snd.to_pitch(pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)
        pitch_values = pitch.selected_array['frequency']
        pitch_values[pitch_values == 0] = np.nan
        mean_voiced_f0 = np.nanmean(pitch_values)
        return mean_voiced_f0 if not np.isnan(mean_voiced_f0) else None
    except Exception as e:
        print(f"F0抽出中にエラー: {e}")
        return None

def estimate_height_from_f0(f0: float, selected_gender: str) -> float:
    """F0 (Hz) から身長 (cm) を推定します。"""
    if selected_gender == "男性 (Male)":
        m, c = -0.2237, 196.72  # Hatano et al. ベース
    else: # 女性 (Female)
        m, c = -0.1500, 189.50  # 仮定の統計値ベース
    return (m * f0) + c

# --- モデル2: VTL（声道長）アプローチ ---

def get_mean_formants(snd: parselmouth.Sound, selected_gender: str) -> tuple[float, ...] | None:
    """
    有声区間のみから平均フォルマント (F1, F2, F3, F4) を抽出します。
    """
    if selected_gender == "男性 (Male)":
        max_formant = 5000.0
        pitch_floor, pitch_ceiling = 75.0, 300.0
    else: # 女性 (Female)
        max_formant = 5500.0
        pitch_floor, pitch_ceiling = 150.0, 500.0

    try:
        # F0抽出（有声区間の特定のため）
        pitch = snd.to_pitch(pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)

        # (time_step, max_number_of_formants, maximum_formant, window_length, pre_emphasis_from)
        # の5つの引数を、順番通りに「位置引数」として渡す必要があります。
        # time_step は None ではなく、デフォルト値 0.0 を明示的に指定します。

        formants = snd.to_formant_burg(
            None,         # 1. time_step (デフォルトの None を使用)
            5.0,          # 2. max_number_of_formants (float)
            max_formant,  # 3. maximum_formant
            0.025,        # 4. window_length (デフォルト)
            50.0          # 5. pre_emphasis_from (デフォルト)
        )

        f1_vals, f2_vals, f3_vals, f4_vals = [], [], [], []

        # Pitchオブジェクトの時間ステップをループ
        for t in pitch.ts():
            if pitch.get_value_at_time(t) > 0: # 有声区間（F0 > 0）か？
                # 有声区間であれば、その時刻のF1-F4をリストに追加
                f1_vals.append(formants.get_value_at_time(1, t))
                f2_vals.append(formants.get_value_at_time(2, t))
                f3_vals.append(formants.get_value_at_time(3, t))
                f4_vals.append(formants.get_value_at_time(4, t))

        if not f1_vals: # 有声区間が全く見つからなかった場合
            print("  VTLモデル: フォルマントを抽出可能な有声区間が見つかりませんでした。")
            return None

        # nanを無視して平均を計算
        mean_f1 = np.nanmean(f1_vals)
        mean_f2 = np.nanmean(f2_vals)
        mean_f3 = np.nanmean(f3_vals)
        mean_f4 = np.nanmean(f4_vals)

        if np.isnan(mean_f1) or np.isnan(mean_f2) or np.isnan(mean_f3) or np.isnan(mean_f4):
            print("  VTLモデル: フォルマントの計算中にNaNが発生しました。")
            return None

        return (mean_f1, mean_f2, mean_f3, mean_f4)

    except Exception as e:
        print(f"フォルマント抽出中に予期せぬエラー: {e}")
        traceback.print_exc() # エラーの詳細をすべて表示
        return None

def estimate_vtl_from_formants(f1: float, f2: float, f3: float, f4: float) -> float:
    """
    F1-F4 から 声道長 (VTL, cm) を推定します。
    (単純な管モデルの平均化に基づく)
    """
    c = 34300.0  # 音速 (cm/s)

    vtl1 = (1 * c) / (4 * f1)
    vtl2 = (3 * c) / (4 * f2)
    vtl3 = (5 * c) / (4 * f3)
    vtl4 = (7 * c) / (4 * f4)

    mean_vtl = np.nanmean([vtl1, vtl2, vtl3, vtl4])
    return mean_vtl

def estimate_height_from_vtl(vtl: float) -> float:
    """
    声道長 (VTL, cm) から身長 (cm) を推定します。
    (González & Padrón, 2012 の回帰式に基づく)
    """
    return (6.4 * vtl) + 68.0

# --- 実行パイプライン ---
if 'original_audio_path' in locals() and os.path.exists(original_audio_path):
    print("--- 処理開始 ---")

    # 1. 音声ファイルを変換
    print("ステップA: 音声ファイルを 44.1kHz, 16bit, モノラル に変換中...")
    converted_audio_path = convert_audio_to_target_format(original_audio_path)

    if converted_audio_path:
        snd = parselmouth.Sound(converted_audio_path)
        print("\nステップB: F0（声の高さ）モデルを分析中...")

        # --- モデル1: F0アプローチ ---
        actual_f0 = get_robust_f0(snd, gender)
        if actual_f0:
            estimated_height_f0 = estimate_height_from_f0(actual_f0, gender)
            print(f"  抽出された平均F0: {actual_f0:.2f} Hz")
        else:
            print("  F0モデル: F0（有声区間）を検出できませんでした。")
            actual_f0, estimated_height_f0 = None, None

        # --- モデル2: VTLアプローチ ---
        print("\nステップC: VTL（声道長・フォルマント）モデルを分析中...")
        mean_formants = get_mean_formants(snd, gender)
        if mean_formants:
            f1, f2, f3, f4 = mean_formants
            vtl = estimate_vtl_from_formants(f1, f2, f3, f4)
            estimated_height_vtl = estimate_height_from_vtl(vtl)
            print(f"  抽出された平均フォルマント (F1-F4): {f1:.1f}, {f2:.1f}, {f3:.1f}, {f4:.1f} Hz")
            print(f"  推定された声道長 (VTL): {vtl:.2f} cm")
        else:
            print("  VTLモデル: フォルマントを検出できませんでした。")
            vtl, estimated_height_vtl = None, None

        # --- 最終結果の表示 ---
        print("\n--- 最終推定結果 ---")
        print(f"入力性別: {gender}")

        print("\n[モデル1: F0（声の高さ）モデル]")
        if actual_f0:
            print(f"  抽出F0: {actual_f0:.2f} Hz")
            print(f"  推定身長: {estimated_height_f0:.2f} cm")
            print("  (※ このモデルは発声の癖や話し方の影響を受けやすいです)")
        else:
            print("  分析不可")

        print("\n[モデル2: VTL（声道長・響き）モデル]")
        if vtl:
            print(f"  推定VTL: {vtl:.2f} cm")
            print(f"  推定身長: {estimated_height_vtl:.2f} cm")
            print("  (※ このモデルは物理的な声道長（ハードウェア）に基づいています)")
        else:
            print("  分析不可")

    # クリーンアップ
    if 'original_audio_path' in locals() and os.path.exists(original_audio_path):
        os.remove(original_audio_path)
    if 'converted_audio_path' in locals() and os.path.exists(converted_audio_path):
        os.remove(converted_audio_path)
else:
    print("エラー: ファイルがアップロードされていません。セル2を先に実行してください。")

# **参考文献**
本ツールで採用している2つの推定モデルは、以下の音響音声学における研究および基本原理に基づいています。



[1]
F0モデル（声の高さ）F0（基本周波数）と身長の相関については、ご提示いただいた以下の研究の統計値（日本人男性15名、相関係数 $r = -0.61$）を基に、回帰モデルを構築しました。

 Hatano, H., Kitamura, T., Takemoto, H., Mokhtari, P., Honda, K., & Masaki, S. (2012). "Correlation between vocal tract length, body height, formant frequencies, and pitch frequency for the five Japanese vowels uttered by fifteen male speakers". Proceedings of Interspeech 2012.




[2]
VTLモデル（声道長・響き）VTLモデルは、2段階の推定を行っており、それぞれ異なる理論に基づいています。



A. フォルマント周波数 $\rightarrow$ 声道長 (VTL) の推定音声の「響き」を決めるフォルマント周波数 ($F_n$) は、声道（声帯から唇までの管）の長さ（VTL）と音速 ($c$) によって決まるという音響学の基本原理に基づいています。

本コードでは、声道をもっとも単純な「片側が閉じた管（閉管）」としてモデル化する以下の式を用いて、F1〜F4の周波数から平均VTLを逆算しています。

$F_n = \frac{(2n-1)c}{4 \cdot VTL}$Fant, G. (1960).

Acoustic theory of speech production. Mouton.Stevens, K. N. (1998).

Acoustic phonetics. MIT press.



B. 声道長 (VTL) $\rightarrow$ 身長の推定物理的な声道長 (VTL) と身長 (Height) の間には強い正の相関があることが、MRI（磁気共鳴画像）を用いた研究などで示されています。

VTLモデルは、この生理学的な相関関係を利用しています。

Fitch, W. T., & Giedd, J. (1999). "Morphology and development of the human vocal tract: A study using magnetic resonance imaging". Journal of the Acoustical Society of America, 106(3), 1511-1522.